In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import ydata_profiling as pp

In [193]:
df = pd.read_csv('data/car_price_prediction.csv', ).drop(columns='ID')

df.sample(5)

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
377,18826,707,SSANGYONG,REXTON,2011,Jeep,Yes,Diesel,2,186412 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
13160,7056,-,MITSUBISHI,Colt,2003,Hatchback,No,Petrol,1.3,178000 km,4.0,Automatic,Front,04-May,Right-hand drive,Sky blue,5
9978,21012,-,TOYOTA,Prius,2012,Hatchback,No,Hybrid,1.8,78000 km,4.0,Variator,Front,04-May,Left wheel,Red,7
6806,39201,-,TOYOTA,RAV 4,2015,Jeep,No,Petrol,2.5,88000 km,4.0,Tiptronic,4x4,>5,Left wheel,Purple,12
11786,26343,259,CHEVROLET,Volt,2014,Hatchback,No,Plug-in Hybrid,1.4,66000 km,4.0,Automatic,Front,04-May,Left wheel,Silver,10


In [194]:
df.shape

(19237, 17)

In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             19237 non-null  int64  
 1   Levy              19237 non-null  object 
 2   Manufacturer      19237 non-null  object 
 3   Model             19237 non-null  object 
 4   Prod. year        19237 non-null  int64  
 5   Category          19237 non-null  object 
 6   Leather interior  19237 non-null  object 
 7   Fuel type         19237 non-null  object 
 8   Engine volume     19237 non-null  object 
 9   Mileage           19237 non-null  object 
 10  Cylinders         19237 non-null  float64
 11  Gear box type     19237 non-null  object 
 12  Drive wheels      19237 non-null  object 
 13  Doors             19237 non-null  object 
 14  Wheel             19237 non-null  object 
 15  Color             19237 non-null  object 
 16  Airbags           19237 non-null  int64 

In [196]:
cols_to_numeric = ['Price','Levy', 'Engine volume', 'Mileage', 'Cylinders']

for col in cols_to_numeric:
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        print(col) 

Levy
Engine volume
Mileage


In [197]:
df['Levy'].value_counts()

-       5819
765      486
891      461
639      410
640      405
        ... 
3156       1
2908       1
1279       1
1719       1
1901       1
Name: Levy, Length: 559, dtype: int64

In [198]:
df['Levy'] = pd.to_numeric(df['Levy'].replace('-', np.nan).str.strip())

df['Levy'] = df['Levy'].fillna(0)

In [199]:
df['Engine volume'].unique()

array(['3.5', '3', '1.3', '2.5', '2', '1.8', '2.4', '4', '1.6', '3.3',
       '2.0 Turbo', '2.2 Turbo', '4.7', '1.5', '4.4', '3.0 Turbo',
       '1.4 Turbo', '3.6', '2.3', '1.5 Turbo', '1.6 Turbo', '2.2',
       '2.3 Turbo', '1.4', '5.5', '2.8 Turbo', '3.2', '3.8', '4.6', '1.2',
       '5', '1.7', '2.9', '0.5', '1.8 Turbo', '2.4 Turbo', '3.5 Turbo',
       '1.9', '2.7', '4.8', '5.3', '0.4', '2.8', '3.2 Turbo', '1.1',
       '2.1', '0.7', '5.4', '1.3 Turbo', '3.7', '1', '2.5 Turbo', '2.6',
       '1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.8', '0.2 Turbo', '5.7',
       '4.8 Turbo', '4.6 Turbo', '6.7', '6.2', '1.2 Turbo', '3.4',
       '1.7 Turbo', '6.3 Turbo', '2.7 Turbo', '4.3', '4.2', '2.9 Turbo',
       '0', '4.0 Turbo', '20', '3.6 Turbo', '0.3', '3.7 Turbo', '5.9',
       '5.5 Turbo', '0.2', '2.1 Turbo', '5.6', '6', '0.7 Turbo',
       '0.6 Turbo', '6.8', '4.5', '0.6', '7.3', '0.1', '1.0 Turbo', '6.3',
       '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo', '6.4',
       '3

In [200]:
df[['Engine volume', 'Turbo']] = df['Engine volume'].str.split(' ', expand=True)
df['Engine volume'] = pd.to_numeric(df['Engine volume'].str.strip())


df['Turbo'] = df.Turbo.fillna(0).replace('Turbo',1).astype('category')

In [201]:
df.Mileage.str[-2:].unique()

array(['km'], dtype=object)

In [202]:
df['Mileage'] = pd.to_numeric(df.Mileage.str.replace('km', '').str.strip())

In [203]:
df.Doors.unique()

array(['04-May', '02-Mar', '>5'], dtype=object)

In [204]:
doors_map = {'04-May':4, '02-Mar':2, '>5':5}

df['Doors'] = df.Doors.apply(lambda x: doors_map[x]).astype('category')

In [205]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Price             19237 non-null  int64   
 1   Levy              19237 non-null  float64 
 2   Manufacturer      19237 non-null  object  
 3   Model             19237 non-null  object  
 4   Prod. year        19237 non-null  int64   
 5   Category          19237 non-null  object  
 6   Leather interior  19237 non-null  object  
 7   Fuel type         19237 non-null  object  
 8   Engine volume     19237 non-null  float64 
 9   Mileage           19237 non-null  int64   
 10  Cylinders         19237 non-null  float64 
 11  Gear box type     19237 non-null  object  
 12  Drive wheels      19237 non-null  object  
 13  Doors             19237 non-null  category
 14  Wheel             19237 non-null  object  
 15  Color             19237 non-null  object  
 16  Airbags           1923

In [206]:
pp.ProfileReport(df, minimal=True)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [207]:
df['Final_Price'] = df['Price']+df['Levy']

df = df.drop(columns=['Price', 'Levy'])

In [208]:
numeric_cols = df.select_dtypes('number').columns

In [209]:
numeric_cols

Index(['Prod. year', 'Engine volume', 'Mileage', 'Cylinders', 'Airbags',
       'Final_Price'],
      dtype='object')

In [210]:
df

,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags,Turbo,Final_Price
0,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4x4,4,Left wheel,Silver,12,0,14727.0
1,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3.0,192000,6.0,Tiptronic,4x4,4,Left wheel,Black,8,0,17639.0
2,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000,4.0,Variator,Front,4,Right-hand drive,Black,2,0,8467.0
3,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966,4.0,Automatic,4x4,4,Left wheel,White,0,0,4469.0
4,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901,4.0,Automatic,Front,4,Left wheel,Silver,4,0,12172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,MERCEDES-BENZ,CLK 200,1999,Coupe,Yes,CNG,2.0,300000,4.0,Manual,Rear,2,Left wheel,Silver,5,1,8467.0
19233,HYUNDAI,Sonata,2011,Sedan,Yes,Petrol,2.4,161600,4.0,Tiptronic,Front,4,Left wheel,Red,8,0,16512.0
19234,HYUNDAI,Tucson,2010,Jeep,Yes,Diesel,2.0,116365,4.0,Automatic,Front,4,Left wheel,Grey,4,0,26944.0
19235,CHEVROLET,Captiva,2007,Jeep,Yes,Diesel,2.0,51258,4.0,Automatic,Front,4,Left wheel,Black,4,0,6619.0
